In [6]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
publico_path = '../extra/notícias/news_sample_1to1000_updated.csv'
observador_path = '../extra/notícias/noticias_observador_fev_a_junho.csv'

In [15]:
df_publico = pd.read_csv(publico_path, index_col='Unnamed: 0')
df_publico.index.set_names('index', inplace=True)
df_observador = pd.read_csv(observador_path, index_col='index').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [16]:
df_publico.head()

,titulo,descricao,titulo_Mobile,subtitulo,rubricTag,rubrica,tipo,palavraChave,tipoLayout,isOpinion,...,isClubeP,isItemOpinion,isMultipleRubric,socialTitle,isLongForm,isHeadline,wordCount,extra4,partners,props
index,,,,,,,,,,,,,,,,,,,,,
0,Linha de mil milhões para micro e pequenas emp...,Ministério da Economia diz que a formalização ...,NaN,NaN,NaN,Covid-19,NOTICIA,NaN,MULTIMEDIA_NORMAL,False,...,False,False,False,Covid-19. Linha de mil milhões para micro e pe...,False,False,349,NaN,NaN,NaN
1,Covid-19: Twitter retira vídeo e mensagens de ...,"Segundo o jornal <i>The Washington Post</i>, o...",NaN,NaN,NaN,Coronavírus,NOTICIA,NaN,MULTIMEDIA_NORMAL,False,...,False,False,False,Coronavírus. Covid-19: Twitter retira vídeo e ...,False,False,358,NaN,NaN,NaN
2,Tecnologia portuguesa elimina o novo coronavír...,"A tecnologia pioneira AT MicroProtect, desenvo...",NaN,NaN,NaN,Coronavírus,NOTICIA,NaN,MULTIMEDIA_NORMAL,False,...,False,False,False,Coronavírus. Tecnologia portuguesa elimina o n...,False,False,740,NaN,NaN,NaN
3,"“Não somos super-homens, somos humanos”: os re...",Há de tudo: a algumas pessoas a covid-19 não p...,NaN,NaN,NaN,Coronavírus,NOTICIA,NaN,LONGFORM_NORMAL,False,...,False,False,False,"Coronavírus. “Não somos super-homens, somos hu...",True,False,2108,NOTIFIED,NaN,NaN
4,Dez milhões de euros para as instituições part...,"Da aquisição de máscaras, luvas e viseiras à c...",NaN,NaN,NaN,Covid-19,NOTICIA,NaN,MULTIMEDIA_NORMAL,False,...,False,False,False,Covid-19. Dez milhões de euros para as institu...,False,False,330,NaN,NaN,NaN


In [17]:
df_observador.head()

,Data,Titulo,Desc,Links
index,,,,
0,2020-02-01,Coronavírus. Portugueses ficam 14 dias ...,Os 18 portugueses vindos de Wuhan já aterraram...,https://observador.pt/2020/02/01/aviao-que-vai...
1,2020-02-01,"Wuhan Coronavírus: afinal, qual o risco?",Os coronavírus é uma família de vírus com prov...,https://observador.pt/opiniao/2019-ncov-ou-wuh...
2,2020-02-01,Como o coronavírus afeta fabricantes de...,"Com mais habitantes do que Portugal, Wuhan é u...",https://observador.pt/2020/01/31/como-o-corona...
3,2020-02-01,Centenas são repatriados a partir de Wuhan,Vários países já estão a repatriar os seus cid...,https://observador.pt/programas/atualidade/cen...
4,2020-02-01,Vídeo mostra a origem do coronavírus?,Vídeo que alegadamente mostra um mercado na Ch...,https://observador.pt/factchecks/fact-check-vi...


In [19]:
print('Público:    ', df_publico.shape)
print('Observador: ', df_observador.shape)

Público:     (9670, 42)
Observador:  (12543, 4)
